In [1]:
import glob
import configparser
import os
import csv

config = configparser.ConfigParser()
config.read('config.ini')
input_folder = config['DEFAULT']['Input-Folder']

In [2]:
# read files from a folder
def read_file(input_folder):
    extension = 'csv'
    all_filenames = [i for i in glob.glob(input_folder + '*.{}'.format(extension))]
    return all_filenames

In [3]:
# get dictionary for hashtag and count
def get_dic_hashtag(file,dic_hashtag):
    with open(file,'r') as csv_file:
            file_reader = csv.reader(csv_file,delimiter=',')
            header = next(file_reader,None)
            for row in file_reader:
                hashtag = row[0]
                count = row[2]
                if hashtag in dic_hashtag :
                    dic_hashtag[hashtag] = int(dic_hashtag[hashtag]) + int(count)
                else:
                    dic_hashtag[hashtag] = int(count)
    return dic_hashtag

In [4]:
# get top 100 hashtags for each csv file
def get_top_100_hashtags(all_filenames):
    for file in all_filenames:
        print("----------------------------------")
        filename=os.path.basename(file)
        print("Processing on file: " + filename)
        dic_hashtag = {}
        dic_hashtag = get_dic_hashtag(file,dic_hashtag)
        top_100_htag = dict(sorted(dic_hashtag.items(), key=lambda x: x[1], reverse=True)[:100])
        print("Top 100 hashtags are done.")
        export_csv(filename,top_100_htag)
        print("----------------------------------")
        

In [5]:
# export to csv
def export_csv(filename,top_100_htag):  
    with open(filename, 'w') as f:
        for key in top_100_htag.keys():
            f.write("%s,%s\n"%(key,top_100_htag[key]))
    print("Top 100 hashtags csv file for " + filename + " is ready.")

In [6]:
# main function
all_filenames = read_file(input_folder)
dic_hashtag = get_top_100_hashtags(all_filenames)

----------------------------------
Processing on file: combined_csv_Australia0.csv
Top 100 hashtags are done.
Top 100 hashtags csv file for combined_csv_Australia0.csv is ready.
----------------------------------
----------------------------------
Processing on file: combined_csv_Other0.csv
Top 100 hashtags are done.
Top 100 hashtags csv file for combined_csv_Other0.csv is ready.
----------------------------------
